1. ========================================================================
얼굴을 인식해서 사각형 테두리 표시하기


 원본 이미지 :
resData/photo1.jpg

프롬프트창에
pip install opencv-python 

opencv : 이미지 처리용 파이썬 인공지능 라이브러리


In [15]:


import cv2

# 입력 파일 지정
image_file = "./resData/photo1.jpg"
# 캐스케이드 파일의 경로 지정
cascade_file = cv2.data.haarcascades + "haarcascade_frontalface_alt.xml"
print('cascade_file', cascade_file)
# 이미지 읽기
image = cv2.imread(image_file)
# 그레이스케일로 변환
image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 얼굴인식을 위한 특징 파일 로드
cascade = cv2.CascadeClassifier(cascade_file)
# 얼굴 인식 실행
face_list = cascade.detectMultiScale(image_gs, scaleFactor=1.1, minNeighbors=1, minSize=(150, 150))
if len(face_list) > 0:
    # 인식한 부분 표시
    print(face_list)
    color = (0, 0, 255)  # 빨간색
    # 얼굴 영역에 테두리 표시
    for face in face_list:
        x, y, w, h = face
        cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness=8)
    # 파일로 출력
    cv2.imwrite("./saveFiles/photol-facedetect01.png", image)
else:
    print("얼굴을 인식할 수 없습니다")



cascade_file c:\workspace\face_detection\.venv\Lib\site-packages\cv2\data\haarcascade_frontalface_alt.xml
[[402 479 354 354]]



위는 얼굴의 위치와 크기를 나타내는 (x, y, width, height) 좌표이다.
saveFiles/photol-facedetect01.png에서 확인이 가능하다


2. ========================================================================
 이미지의 얼굴을 인식해서 모자이크 처리하기

 원본 이미지 :
resData/photo2.jpg

In [ ]:
import cv2
import sys
import re
#입력 파일 지정 (아나콘다 프롬프트에서 실행해야함)

if len(sys.argv) <= 1:
	print("no input file")
	quit()
image_file = "./resData/photo2.jpg"
#image_file 경로에 -mosaic1.jpg를 붙임(저장용)
output_file = re.sub(r'\.jpg|jpeg|PNG$', '-mosaic1.jpg', image_file)
print("output", output_file)
#모자이크 강도
mosaic_rate = 30
#캐스캐이드 파일 경로 지정
cascade_file = cv2.data.haarcascades + "haarcascade_frontalface_alt.xml"
#이미지를 Numpy 배열로 변환
image = cv2.imread(image_file)
#그레이스케일
image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#얼굴 인식 실행하기
cascade = cv2.CascadeClassifier(cascade_file)
face_list = cascade.detectMultiScale(image_gs,
scaleFactor=1.1, minNeighbors=1, minSize=(100,100))
#얼굴이 감지되지 않으면 프로그램 종료
if len(face_list) == 0:
	print("얼굴을 인식할 수 없습니다.")
	quit()
print(face_list)


output ./resData/photo2-mosaic1.jpg
[[599 481 205 205]
 [293 142 239 239]]


위는 파이썬의 선형대수같은 수학 계산을 위한 Numpy(수학용 배열로 만들어줌) 배열이다.
Numpy 라이브러리를 이용한다. (openCV 설치하면 자동 임포트) 

따로 가져오고 싶으면
pip install numpy

In [6]:
#확인한 부분을 모자이크로 처리
color = (0, 0, 255)
for (x,y,w,h) in face_list:
# 얼굴 부분 자르기
	face_img = image[y:y+h, x:x+w]
	# 자른 이미지를 지정한 배율로 확대/축소하기
	face_img = cv2.resize(face_img, (w//mosaic_rate, h//mosaic_rate))
	#확대/축소한 그림을 원래 크기로 돌리기
	face_img = cv2.resize(face_img, (w, h), interpolation=cv2.INTER_AREA)
	# 원래 이미지에 붙이기
	image [y:y+h, x:x+w] = face_img
# 렌더링 결과를 파일에 출력
cv2.imwrite(output_file.replace("resData", "saveFiles"), image)

True

위 내용은 saveFiles/photo2-mosaic1.jpg 에서 확인이 가능하다.
얼굴에 모자이크 처리가 되었다.


3. ========================================================================
이미지 내의 숫자 인식해서 외곽선 표시

원본 이미지 resData/numbers.png

In [7]:
import cv2

# 숫자 인식을 위한 이미지 지정
TEST_IMG = ['numbers.png', 'numbers_contour.png']

# 이미지 로드
im = cv2.imread(f'./resData/{TEST_IMG[0]}')

# 그레이스케일로 변환
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# 가우시안 블러를 적용하여 노이즈를 제거
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# 적응형 이진화 적용
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# 윤곽 추출하기
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# 추출한 윤곽을 반복 처리하기
for cnt in contours:
    # 윤곽을 감싸는 바운딩 박스 좌표 계산
    x, y, w, h = cv2.boundingRect(cnt)
    # 높이가 20 미만이면 무시
    if h < 20:
        continue
    red = (0, 0, 255)
    # 바운딩 박스를 빨간색으로 표시
    cv2.rectangle(im, (x, y), (x + w, y + h), red, 2)

# 윤곽선을 표시한 이미지 저장
cv2.imwrite(f'./saveFiles/{TEST_IMG[1]}', im)
print("Task Finished..!!")

Task Finished..!!
